In [869]:
import cplex
from cplex.exceptions import CplexError
import sys
import pandas as pd
import numpy as np
import docplex

In [870]:
def pprint(constraints: list, names: list):
    for c in constraints:
        _c = [str(_i) + "*" + n if _i != 0 else "0" for _i, n in zip(c[0], names)]
        print(c[3] + ": ( [" + " ".join(_c) + "]  " + c[1] + "  " + str(c[2]) + " )") 
        

In [891]:
xls = pd.ExcelFile('logistic.xlsx')

vehicles = [1,2,3,4,5]                                # V
customers = [1,2,3,4,5,6,7,8,9]                       # C

num_vehicles = 2
num_customers = 4
num_locations = num_customers + 1                     
num_names = (num_locations ** 2) * num_vehicles

locations = [0] + customers[:num_customers]           # C': gồm các customers và Depot suất phát + Depot kết thúc

# quy ước: 
#    + (i,j) là cạnh (edge) thuộc tập E
#    + i,j là 2 customer liền kề trên 1 route

In [892]:
my_names = ["x"+str(i)+str(j) + str(k) for k in range(1,num_vehicles + 1) for i in range(0,num_locations) for j in range(0, num_locations) ]  

print(len(my_names))
print("Names ", my_names)
def display(constraints):
    pprint(constraints, my_names)

50
Names  ['x001', 'x011', 'x021', 'x031', 'x041', 'x101', 'x111', 'x121', 'x131', 'x141', 'x201', 'x211', 'x221', 'x231', 'x241', 'x301', 'x311', 'x321', 'x331', 'x341', 'x401', 'x411', 'x421', 'x431', 'x441', 'x002', 'x012', 'x022', 'x032', 'x042', 'x102', 'x112', 'x122', 'x132', 'x142', 'x202', 'x212', 'x222', 'x232', 'x242', 'x302', 'x312', 'x322', 'x332', 'x342', 'x402', 'x412', 'x422', 'x432', 'x442']


In [893]:
cost_matrix = pd.read_excel(xls, 'Cost Matrix')
my_obj = [int(cost_matrix.iat[i,j]) for i in range(0,num_locations ) for j in range(1,num_locations + 1)]*num_vehicles

print("Object ",my_obj)

Object  [0, 18, 30, 28, 27, 18, 0, 23, 10, 4, 30, 23, 0, 32, 29, 28, 10, 32, 0, 2, 27, 4, 29, 2, 0, 0, 18, 30, 28, 27, 18, 0, 23, 10, 4, 30, 23, 0, 32, 29, 28, 10, 32, 0, 2, 27, 4, 29, 2, 0]


In [894]:
capacity_matrix = pd.read_excel(xls, 'Capicity')
vehicles_capacity = [capacity_matrix.iat[i,1] for i in range(0, num_vehicles)]
vehicles_capacity

[100, 70]

In [895]:
demand_matrix = pd.read_excel(xls, 'Demand Matrix')
demand = {customers[i]: demand_matrix.iat[i,1] for i in range(0, num_customers)}
demand

{1: 32, 2: 45, 3: 16, 4: 15}

In [896]:
lower_bounds = [0]*num_names
upper_bounds = [1]*num_names

print("Lb ", lower_bounds)
print("Rb ", upper_bounds)

Lb  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Rb  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [897]:
# CONSTRAINTS_2: 
# # Một cạnh bất kỳ (i->j) chỉ tổn tại trên duy nhất 1 Vehicle. Nhưng i CHỈ LÀ customers, j là locations
contrains_2 = []
_i = 0
for i in locations[:num_locations]:
    for j in locations[:num_locations]:
        con_ = [(1 if i != j and int(name[1]) == i and int(name[2]) == j else 0) for name in my_names]
        if len([c for c in con_  if c != 0]) > 0:
            _i += 1

            contrains_2.append((con_,"E", 1, "C2" + str(_i)))
    
display(contrains_2)


C21: ( [0 1*x011 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x012 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C22: ( [0 0 1*x021 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x022 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C23: ( [0 0 0 1*x031 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x032 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C24: ( [0 0 0 0 1*x041 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x042 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C25: ( [0 0 0 0 0 1*x101 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C26: ( [0 0 0 0 0 0 0 1*x121 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x122 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C27: ( [0 0 0 0 0 0 0 0 1*x131 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x132 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C28: ( [0 0 0 0 0 0 0 0 0 1*x141 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x142 0 0 0 0 0 0 0 0 0 0 0 0

In [898]:
# CONSTRAINTS_3: 
# # Lượng hàng chở đển Customer của 1 Vehicle phải nhỏ hơn bằng Capacity. Nhưng i CHỈ LÀ customers, j là locations
contrains_3 = []
_i = 0
for veh, cap in zip(vehicles[0:num_vehicles],vehicles_capacity[0:num_vehicles]): 
    _i += 1
    contrains_3.append(([int(demand[int(name[1])]) if (int(name[3]) == veh and int(name[1]) != 0 and int(name[1]) != int(name[2]) ) else 0 for name in my_names],"L", float(cap), "C3" + str(_i)))
display(contrains_3)

C31: ( [0 0 0 0 0 32*x101 0 32*x121 32*x131 32*x141 45*x201 45*x211 0 45*x231 45*x241 16*x301 16*x311 16*x321 0 16*x341 15*x401 15*x411 15*x421 15*x431 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  L  100.0 )
C32: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 32*x102 0 32*x122 32*x132 32*x142 45*x202 45*x212 0 45*x232 45*x242 16*x302 16*x312 16*x322 0 16*x342 15*x402 15*x412 15*x422 15*x432 0]  L  70.0 )


In [899]:
# CONSTRAINTS_4: 
# # Với mỗi Vehicle, khi suất phát từ Depot, nó chỉ có thể đến duy nhất 1 Customer
contrains_4 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    _i += 1
    contrains_4.append(([(1 if (int(name[1]) == 0 and int(name[3]) == k) else 0) for name in my_names],"E", 1, "C4" + str(_i)))
    
display(contrains_4)

C41: ( [1*x001 1*x011 1*x021 1*x031 1*x041 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C42: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x002 1*x012 1*x022 1*x032 1*x042 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )


In [900]:
# CONSTRAINTS_5: 
# # Trên route của Vehicle k có cạnh (i->j) : tổng tất cả cạnh đầu 'ra' của i BẰNG tổng tất cả cạnh đầu 'vào' của j
contrains_5 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    for p in customers[:num_customers]: # i
        _i += 1
        foo = lambda k,p,n: 0 if int(n[3]) != k or int(n[2]) == int(n[1]) else 1 if int(n[1]) == p else -1 if int(n[2]) == p else 0                
        contrains_5.append(([foo(k, p, name) for name in my_names],"E", 0, "C5" + str(_i)))

# for k in vehicles[0:num_vehicles]:
#     for p in customers[:num_customers]: # i
#         for h in customers[:num_customers]: # j
#             _i += 1
#             foo = lambda k,p,h,n: 0 if int(n[3]) != k or int(n[2]) == int(n[1]) else 1 if int(n[2]) == p else -1 if int(n[1]) == h else 0                
#             contrains_5.append(([foo(k, p, h, name) for name in my_names],"E", 0, "C5" + str(_i)))

display(contrains_5)

C51: ( [0 -1*x011 0 0 0 1*x101 0 1*x121 1*x131 1*x141 0 -1*x211 0 0 0 0 -1*x311 0 0 0 0 -1*x411 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  0 )
C52: ( [0 0 -1*x021 0 0 0 0 -1*x121 0 0 1*x201 1*x211 0 1*x231 1*x241 0 0 -1*x321 0 0 0 0 -1*x421 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  0 )
C53: ( [0 0 0 -1*x031 0 0 0 0 -1*x131 0 0 0 0 -1*x231 0 1*x301 1*x311 1*x321 0 1*x341 0 0 0 -1*x431 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  0 )
C54: ( [0 0 0 0 -1*x041 0 0 0 0 -1*x141 0 0 0 0 -1*x241 0 0 0 0 -1*x341 1*x401 1*x411 1*x421 1*x431 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  0 )
C55: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1*x012 0 0 0 1*x102 0 1*x122 1*x132 1*x142 0 -1*x212 0 0 0 0 -1*x312 0 0 0 0 -1*x412 0 0 0]  E  0 )
C56: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1*x022 0 0 0 0 -1*x122 0 0 1*x202 1*x212 0 1*x232 1*x242 0 0 -1*x322 0 0 0 0 -1*x422 0 0]  E  0 )
C57: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 

In [901]:
# CONSTRAINTS_6: 
# # Route của Vehicle k phải luôn trở về Depot (n+1), tức là luôn tồn tại điểm i trên route của k sao cho nó đi đến n+1
# # trong đó n+1 là Depot kết thúc
contrains_6 = []
_i = 0
for veh in vehicles[0:num_vehicles]: 
    _i += 1
    contrains_6.append(([(1 if (int(name[3]) == veh and int(name[2]) == 0 and int(name[1]) != 0) else 0) for name in my_names],"E", 1, "C6" + str(_i)))
    
display(contrains_6)


C61: ( [0 0 0 0 0 1*x101 0 0 0 0 1*x201 0 0 0 0 1*x301 0 0 0 0 1*x401 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]  E  1 )
C62: ( [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1*x102 0 0 0 0 1*x202 0 0 0 0 1*x302 0 0 0 0 1*x402 0 0 0 0]  E  1 )


In [902]:
# CONSTRAINTS_X:
contrains_x = []
_i = 0

_i += 1
contrains_x.append(([(1 if int(name[1]) == int(name[2]) else 0) for name in my_names],"E", 0, "CX" + str(_i)))

display(contrains_x)


CX1: ( [1*x001 0 0 0 0 0 1*x111 0 0 0 0 0 1*x221 0 0 0 0 0 1*x331 0 0 0 0 0 1*x441 1*x002 0 0 0 0 0 1*x112 0 0 0 0 0 1*x222 0 0 0 0 0 1*x332 0 0 0 0 0 1*x442]  E  0 )


In [903]:
# # CONSTRAINT_Y
# _i = 0
# for 
# contrains_x.append(([(1 if int(name[1]) == int(name[2]) else 0) for name in my_names],"E", 0, "CX" + str(_i)))

In [904]:
# Merge segment constraints
constraints_group = []
# constraints_group += contrains_2 
# constraints_group += contrains_3 
# constraints_group += contrains_4 
# constraints_group += contrains_5 
constraints_group += contrains_6
constraints_group += contrains_x



# Convert for cplex
constraints_ = []
constraint_senses = []
rhs = []
constraint_names = []
for i in constraints_group:
    constraints_.append([my_names, i[0]])
    constraint_senses.append(i[1])
    rhs.append(i[2])
    constraint_names.append(i[3])
    
    
# print(constraints_)
# print(constraint_senses)
# print(rhs)

In [905]:
# Create Cplex Problems
myProblem = None

try:
    myProblem = cplex.Cplex()     
except CplexError:
    print ("exc")


myProblem.variables.add(obj = my_obj,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = my_names)

myProblem.objective.set_sense(myProblem.objective.sense.minimize)

for i in range(0, num_names):
    myProblem.variables.set_types(i,myProblem.variables.type.binary)
    
myProblem.linear_constraints.add(lin_expr = constraints_,
                                 senses = constraint_senses,
                                 rhs = rhs,
                                 names = constraint_names)

range(0, 3)

In [906]:
# Find the solutions
try:
    myProblem.solve()
    
    print(myProblem.get_stats())
    values=myProblem.solution.get_values()
    names=myProblem.variables.get_names()
    
    for v, n in zip(values, names):
        if v != 0:
            print(n, " - ",v)
except cplex.exceptions.errors.CplexSolverError:
    print("No solution")

Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 36.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 3 rows and 50 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.02 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.02 ticks)
Problem name         : 
Objective sense      : Minimize
Variables            :      50  [Binary: 50]
Objective nonzeros   :      40
Linear constraints   :       3  [Equal: 3]
  Nonzeros           :      18
  RHS nonzeros       :       2

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzeros   : Min   : 2.000000    